# 决策树基本理论和简单算法

> 本文档主要就目前简单的决策树原理和算法进行简单介绍
> 其中包括**决策树，集成学习中的Bagging：随机森林 和 Boosting：GBDT**

基本参考的是**numpy-ml**中内容进行加工整理，写这个的目的在于通过自己的角度对整个算法进行梳理

写在最前面的话：从陈天奇对xgboost进行解释的文章中，提到了机器学习的主要组建：**模型，参数，目标函数**，本文档打算在此基础上进行阐述，本质上而言就是在说**用什么模型**，模型是有很多**参数**构成，但是我们还不知道**如何**去确定这些参数，因此需要**目标函数**这种**评价的手段**进行寻找参数来找到我们认为比较不错的模型

## 理论基础

(1). 关于什么是决策树？简单直观点看的话就是人们对于“如果。。。。。。那么的”程序化利用

(2). 这里的**模型**显然就是**决策树**。**目标**如何从找到好的决策树，这个好就是指的决策树跑完之后我的叶子节点的数据很多都是具有一样的属性或者‘纯度’，这个时候的好坏评判标准就会有差异如果目标变量是**离散值**则可以为信息熵增益最大（ID3），信息熵增益速率（C4.5），gini不纯度来衡量（CART）如果目前变量是**连续值**，则可以用均方误差来衡量，无论采用何种指标最后都会回到**不纯度的衰减幅度上**


$\Delta(L) = L(parent) - P_{left}*L(Left\quad child) - P_{right}*L(Right\quad child)$

差别就是我们这个纯度的不同定义罢了，其中$P_{left}$指的是相应叶节点占的总的比例

对于**离散值**定义有信息熵增益和基尼系数的评价指标

**其中信息熵**定义为$-\sum(P(w))logP(w)$ 

**其中gini系数**定义为$-\sum(P(w)*(1-P(w)) = 1 - \sum(P^2(w))$ 有一种解释是gini系数是信息熵的一阶泰勒展开

> 为什么要用gini系数，因为gini系数没有涉及到对数的运算，这样处理简化了模型


(3). 决策树的过程：我们需要找到最佳的分割点进行分割，然后继续分割。因为算法的参数包括，**分割的变量，和分割点**

20世纪80年代澳大利亚科学家昆兰将信息熵引入了决策树之后，决策树理论才算做有了比较长足的进步和发展，昆兰首先提出了ID3的算法，**只处理离散值**，而且这里的决策树是**不是二叉树**，原文中就有三个叶节点，C4.5是ID3的改进版本，可以处理连续值。CART树的是采用**二叉树**的结构，可以处理离散值（gini）和连续值（mse），其处理连续值是叶子的值可以用平均值来表示。这是这些算法的主要区别

## 理论发展：随机森林，GBDT，XGBOOST

**随机森林**：有放回的从总样本的进行抽样，使用决策数进行决策，最后选择**投票或者均值**来进行决策

**GBDT**：简单而言通过像练习高尔夫球一样去，将上次决策树训练的结果差距作为输入进行下一次（颗）决策树的训练，通过一阶的泰勒的展开来寻找参数，通过将所有决策树结果进行**求和**的运算来进行对函数进行逼近，也就是

$\sum_{k=1}^{K}f_k(X_i)$

**XGBOOST**：GBDT的高阶版本，引入了正则项和二阶泰勒展开

对于这三种算法其实都是集成学习的范畴，就是我有很多学习器进行学习，如何组织这些学习器便形成了我们所说的**bagging**和**boosting**，代表分别就是**随机森林**和**GBDT，XGBOOST**，从学习器的组织方式来看前者属于**并行**后者属于**串行**，从偏差和方差的角度看，前者属于**高偏差，低方差**，因为学习器都是独立的组合，后者属于**低偏差，高方差**因为后面的学习器的输入是前面的输出


## 1.决策树算法的代码实现

(1). 由于我们首先提到了信息熵，gini，mse等评价指标我们可以定义如下的函数


In [1]:
def entropy(y):
    hist = np.bincount(y)
    ps = hist/len(y)
    return -np.sum(p*np.log(p) for p in ps if p> 0)

def gini(y):
    hisr = np.bincount(y)
    ps = hist/len(y)
    return 1- np.sum(p**2 for p in ps)

def mse(y):
    return np.mean((y - np.mean(y))**2)


(2). 具体运算过程，涉及类的处理，回头看类的定义，虽然不见的这样写类有多么的优雅但是其中类的嵌套和递归的方法还是值的我自己去好好看和学习的

In [ ]:
import numpy as np

class Node:
    def __init__(self,left,right,rule):
        self.left = left
        self.right = right
        self.feature = rule[0]
        self.threshold = rule[1]

class Leaf:
    def __init__(self,value):
        self.value = value
        
#定义好了节点和叶子之后开始进行决策树类的编写，涉及到许多方法，这段对于我而言其实有点搞
#对于决策数类的定义这块，可以考虑流程化的角度去看，刚才定义的节点和叶子的时候，其实有点像那种最小化知识的思考
#对于刚开始的决策树设计流程而言的话，就假设我是开始弄决策树了，我刚开始的深度肯定为0，根节点还没有。为了开始进行树的生长，我需要选择选择选择的变量，
#按照什么标准进行分裂，最大深度是多少，计算叶子的时候，是按照目标是分类变量还是连续值，连续值的话可输出值，离散值的话输出概率，基本就是这思路，如果这样
#想的话其实整个程序的思路还是比较清楚的，说白了这个的初始化就是说
class DecisionTree:
    def __init__(self,classifier =True,max_depth=None,n_feats=None,criterion='entropy',seed=None,):
        if seed:
            np.random.seed(seed)
        
        self.depth = 0
        self.root = None
        
        self.n_feats = n_feats
        self.criterion = criterion
        self.classifier = classifier
        self.max_depth = max_depth if max_depth else np.inf
        
        if not classifier and criterion in ['gini','entropy']:
            raise ValueError(
                "{} is a valid criterion only when classifier = True.".format(criterion)
            )
        if classifier and criterion == 'mse':
            raise ValueError("'mse' is a valid criterion only when classifier = Flase.")
            
    def fit(self,X,Y):
        self.n_classes = max(Y) + 1 if self.classifier else None
        self.n_feats = X.shape[1] if not self.n_feats else min(self.n_feats,X.shape[1])
        self.root=self._grow(X,Y)
        
    def predict(self,X):
        return np.array([self._traverse(x,self.root) for x in X])
    
    def predict_class_probs(self,X):
        assert self.classifier,"'predict_calss_probs' undefined for classifier = False"
        return np.array([self._traverse(x,self.root,prob=True) for x in X])
#下面的函数才基本上是整个决策树的关键，涉及到了树的生长分割，计算信息增益等基本的计算，不过由于类的饮用稍微还是有点绕
#在写这个程序的时候发现作者有个习惯，就是喜欢将那些异常的特殊的情况进行讨论，然后再看正常的情况
#要看数据的生长那先看那些情况下我们不进行分割，一个是我分组比较完美，可以分的干净，另一个是达到我最大的学习深度了
#这个grow的数据结构是什么呢？目前有点看的不是很懂，或者说返回的值是怎样的结构。因为我的理解这个node只是包含了最后叶节点的分割点和阈值，
#那之前的信息在哪里呢？关于这个点我自己表示还没有理解清楚
    def _grow(self,X,Y):
        if len(set(Y))==1:
            if self.classifer:
                prob = np.zeros(self.n_classes)
                prob[Y[0]] = 1.0
            return Leaf(prob) if self.classifer else Leaf(Y[0])
        
        if self.depth >= self.max_depth:
            v = np.mean(Y,axis=0)
            if self.classifier:
                v = np.bincount(Y,minlength=self.n_classes) / len(Y)
            return Leaf(v)
#特殊的情况处理好了之后便开始进行树的生长        
        N,M = X.shape
        self.depth += 1
        feat_idxs = np.random.choice(M,self.n_feats,replace=True)
        
        feat,thresh = self._segment(X,Y,feat_idxs)
        l = np.argwhere(X[:,feat] <= thresh).flatten()
        r = np.argwhere(X[:,feat] > thresh).flatten()
        
#这个转换好不错，我还反应了下，个人觉得好不错,而且还是类的递归
        left = self._grow(X[l,:],Y[l])
        right = self._grow(X[r,:],Y[r])
        return Node(left,right,(feat,thresh))

    def _segment(self,X,Y,feat_idxs):
        best_gain = -np.inf
        split_idx,split_thresh = None,None
        for i in feat_idxs:
            vals = X[:,i]
            levels = np.unique(vals)
            thresholds = (levels[:-1]+levels[1:])/2 
            gains = np.array([self._impurity_gain(Y,t,vals) for t in thresholds])
#关于为什么要这样设置阈值，我都不知道为什么，不过也可以说出个大概来，就是对特征变量的值进行去重排序，取中间值进行阈值的筛选
            if gains.max() > best_gain:
                split_idx = i
                best_gain = gains.max()
                split_thresh = thresholds[gains.argmax()]
        return split_idx,split_thresh
#计算信息增益，分解下来就是简单的母节点的信息熵 - 叶节点的信息熵    
    def _impurity_gain(self,Y,split_thresh,feat_values):
        if self.criterion == "entropy":
            loss = entropy
        elif self.criterion == "gini":
            loss = gini
        elif self.criterion == "mse":
            loss = mse
        
        parent_loss = loss(Y)
        
        left = np.argwhere(feat_values <= split_thresh).flatten()
        right = np.argwhere(feat_values > split_thresh).flatten()
        
        if len(left) == 0 or len(right) == 0:
            return 0
#为啥返回0 呢？我也有点纳闷，增益为0？
        
        n = len(Y)
        n_l,n_r = len(left),len(right)
        e_l,e_r = loss(Y[left]),loss(Y[right])
        child_loss = (n_l/n) * e_l + (n_r/n)*e_r
        
        ig = parent_loss -child_loss
        
        return ig
# 这里的value是指的叶子节点？这个递归也也太多了吧
    def _traverse(self,X,node,prob = False):
        if isinstance(node,Leaf):
            if self.classifier:
                return node.value if prob else node.value.argmax()
            return node.value
        if X[node.feature] <= node.threshold:
            return self._traverse(X,node.left,prob)
        return self._traverse(X,node.right,prob)
# 这个traversek看的我比较迷，表示需要这个吗
# 看的有点那么懂了    

    
        #先从简单的代码进行处理，例如定义函数的评估函数
    def mse(y):
        return np.mean((y - np.mean(y))**2)

    def entropy(y):
        hist = np.bincount(y)
        ps = hist /np.sum(hist)
        return  -np.sum([p*np.log2(p) for p in ps if p > 0])
    def gini(y):
        hist = np.bincount(y)
        ps = hist/np.sum(hist)
        return 1- np.sum([i**2 for i in ps])

## 2. 随机森林的算法实现

由刚才决策树的实现代码来进行随机森林代码实现

In [ ]:
# 随机森林的编写，用到了决策树

import numpy as np
from .dt import DecisionTree

# 使用自主法进行有放回的抽样
# 不过我对这个代码中为什么会取全量的抽样比较困惑，虽然那样选择也不是全量因为肯定是有对样本有重复的选择
# 在写类的时候最大的感受是没有感受到面对过程时需要return什么，这点我自己不是那么理的清楚，更多的是写了很多方法之类的

def bootstrap_sample(X,Y):
    N,M = X.shape
    idxs = np.random.choice(N,N,replace=True)
    return X[idxs],Y[idxs]

# 进行随机森林的算法
class RandomForest:
    def __init__(self,n_trees,max_depth,n_feats,classifier=True,criterion = "enropy")
        self.trees = []
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.classifier = classifier
        self.criterion = criterion
        
    def fit(self,X,Y):
        self.trees = []
        for _ in range(self.n_tress):
            X_samp, Y_samp = bootstrap_sample(X,Y)
            tree = DecisionTree(
                n_feats = self.n_feats,
                max_depth=self.max_depth,
                criterion = self.criterion,
                classifier = self.classifier,
            )
            tree.fit(X_samp,Y_samp)
            self.trees.append(tree)
# 这个append的tree是append的啥 这点我自己其实有点懵
# 下面那个调用我自己也有点弄的半懂半不懂的
def predict(self,X):
    tree_peds = np.array([[t._traverse(x,t.root) for x in X] for t in self.trees])
    return self._vote(tree_preds)
# 为什么要转置，表示有点懵
def _vote(self,predictions):
    if self.classifier:
        out = [np.bincount(x).argmax() for x in predictions.T]
    else:
        out = [np.mean(x) for x in prediction.T]
    return np.array(out)

## 3. GBDT的算法实现

鉴于GBDT的算法是对上面决策树和后面的XGBOOST，承上启下作用的算法，因此在这里对算法进行较为详细的讲解

主要的参考资料是[GBDT原理介绍](https://www.csuldw.com/2019/07/12/2019-07-12-an-introduction-to-gbdt/),[GBDT最早的论文Friedman](https://statweb.stanford.edu/~jhf/ftp/trebst.pdf)和[知乎一文理解GBDT的原理](https://zhuanlan.zhihu.com/p/29765582) 和[陈天奇对于xgboost的解释](https://homes.cs.washington.edu/~tqchen/pdf/BoostedTree.pdf)

就像刚才讲到的一样GBDT的本质是，所有决策树的结果进行**加权求和**的运算来进行对函数进行逼近，其中模型的是我们的**CART树**，因为我们离散和连续的变量都要进行处理

$\sum_{k=1}^{K}f_k(X_i)$

树的结构和树之间的处理方式都已经知道，参数就是决策树的那些参数，后面就是怎么样去评价每一次增加一颗数对我们的预测有多大的改善，所以我们需要引入**目标函数**其实我更愿意将之看成评估函数，因为本身就是对模型的参数进行评估和选择。

$
Obj^t 
=\sum_{i=1}^{n}l(y_i,\hat{y}_i^{t})
\\=\sum_{i=1}^{n}l(y_i,\hat{y}_i^{t-1}+f_t(X_i))
\\\approx\sum_{i=1}^{n}[l(y_i,\hat{y}_i^{t-1})+\partial_{\hat{y}^{t-1}_{i}} l(y_i,\hat{y}_i^{t-1})f_t(X_i)]
$

上面的目标函数就是我们对于新加入的树进行衡量的标准，其中上标t是指的第t颗树，t-1指的是前面t-1颗树。由于我们现在做的是对新加入对树进行衡量，所以前面t-1颗树或者说前面t-1轮的模型预测结果我们是已知的，因为我们这个目标函数其实就是看真实值和预测值的差距，所以这个目标函数需要尽量小，所谓的“误差小”，除去常数项后便可看到我们的**目标函数只依赖于每个数据点的在误差函数上的一阶导数**，所以这也是我们GBDT中梯度的由来，在陈天奇的xgboost直接将这个公式（当然是二阶的泰勒展开）添加了正则项对算法进行了进一步的改进，而**Friedman的论文是将一阶导的负数当成学习的函数来进行拟合求参数**，下面的代码也是按照那样来的。这点区别自己也可以注意下，虽然后面还有对GBDT的目标函数更进一步的公式简化和推导，但是以上公式是GBDT最为核心的思想，至于为什么用梯度，Friedman说这是最常用和最简单的方法，有点类似gini系数简化计算结果

In [ ]:
import numpy as np 

class ClassProbEstimator:
    def fit(self,X,y):
        self.class_prob = y.sum() / len(y)
        
    def predict(self,X):
        pred = np.empty(X.shape[0],dtype=np.float64)
        pred.fill(self.class_prob)
        return pred
    
class MeanBaseEstimator:
    def fit(self,X,y):
        self.avg = np.mean(y)
        
    def predict(self,X):
        pred = np.empty(X.shape[0],dtype=np.float64)
        pred.fill(self.avg)
        return pred
# 加入了调用函数，将定义的类的实例也当成函数来
# 但是坦白而言没有看到作者后面利用了这个啊，比较困惑那为啥要定义这个
class MSELoss:
    def __call__(self,y,y_pred):
        return np.mean((y - y_pred)**2)
        
    def base_estimator(self):
        return MeanBaseEstimator()
# 这个是弄grad定义成这样主要是为了什么呢？这个我有点不是很理解对于我来说
# 只能说稍微有点知道了，这个计算梯度和我们如何定义损失函数关，其实不是很知道为什么会有len(y),看系数有2应该就是平方和的感觉啊
# 关于len(y)就是那个N的意思，计算的是一个样本的梯度，所以要弄一下
    def grad(self,y,y_pred):
        return -2 / len(y) * (y - y_pred)
# 线性搜索也不是那么的懂，这个h_pred是什么意思
# 这里的一个作用就是在调整步长，和作者所说的是调整weight来的有点出入
# 这里的设定更多的是输入的时候对步长进行的调整，默认其实还是按照恒定的步长
# 懵懵懂不是那么的清楚为什么，friedman的论文中有写到为什么
    def line_search(self,y,y_pred,h_pred):
        Lp = np.sum((y - y_pred) * h_pred)
        Lpp = np.sum(h_pred * h_pred)
        
        return 1 if np.sum(Lpp) == 0 else Lp / Lpp
# 这个eps也看不懂,是machine epsilon 可以理解为计算机在计算存储浮点数不准确，给了一个上限

class CrossEntropyLoss:
    def __call__(self,y,y_pred):
        eps = np.finfo(float).eps
        return -np.sum(y * np.log(y_pred + eps))
     
    def base_estimator(self):
        return ClassProbEstimator()
    
    def grad(self,y,y_pred):
        eps = np.finfo(float).eps
        return -y* 1/(y_pred + eps)
    
    def line_search(self,y,y_pred,h_pred):
        raise NotImplementedError
    
# 整个块为什么要这么定义我自己其实并不是很明白，为什么要这样去弄这样的结构


# 定义好损失函数之后开始写GBDT的函数
# y变量的one_hot编码

import numpy as np

from .dt import DecisionTree
from .losses import MSELoss,CrossEntropyLoss

def to_one_hot(labels,n_classes=None):
    if labels.ndm > 1:
        raise ValueError("labels must have dimension 1,but got {}".format(labels.ndim))
        
    N = labels.size
    n_cols = np.max(labels) + 1 if n_classes is None else n_classes
    one_hot = np.zeros((N,n_cols))
    one_hot[np.arange(N),labels] = 1.0
    return one_hot


# 先上来就定义了一大段感觉也是醉了，这样看的话确实感觉有点尴尬面向对象的语言
class GradientBoostedDecisionTree:
    def __init__(
        self,
        n_iter,
        max_depth = None,
        classifier = True,
        learning_rate = 1,
        loss = "crossentropy",
        step_size = "constant"
    ):
        self.loss = loss
        self.weights = None
        self.learners = None
        self.out_dims = None
        self.n_iter = n_iter
        self.base_estimator = None
        self.max_depth = max_depth
        self.step_size = step_size
        self.classifier = classifier
        self.learning_rate = learning_rate
        
    def fit(self,X,Y):
        if self.loss == "mse":
            loss = MSELoss()
        elif self.loss == "crossentropy":
            loss = CrossEntropyLoss()
# 后面那个为什么要那样的变换，这点我不是很懂为啥
# 这里是对Y进行的变换，如果是分类的话进行one——hot编码，如果是连续变量且纬度为1的话就变成列变量
        if self.classifier:
            Y = to_one_hot(Y.flatten())
        else:
            Y = Y.reshape(-1,1) if len(Y.shape) == 1 else Y
    
        N, M = X.shape
        self.out_dims = Y.shape[1]
        self.learners = np.empty((self.n_iter,self.out_dims),dtype=object)
        self.weights = np.ones((self.n_iter,self.out_dims))
        self.weights[1:,:] *= self.learning_rate
# 下面这些也知道是什么但是还是不懂。。。。。   
        Y_pred = np.zeros((N,self.out_dims))
        for k in range(self.out_dims):
            t = loss.base_estimator()
            t.fit(X,Y[:,k])
            Y_pred[:,k] += t.predict(X)
            self.learners[0,k] = t

        for i in range(1,self.n_iter):
            for k in range(self.out_dims):
                y,y_pred = Y[:,k],Y_pred[:,k]
                neg_grad = -1 * loss.grad(y,y_pred)

                t = DecisionTree(
                   classifier = False,max_depth=self.max_depth,criterion="mse"
                )
#这里的fit中的y就是指的负梯度的方向，这也算是最为精华的部分了，或者点题的部分
                t.fit(X,neg_grad)
                self.learners[i,k] = t
# 
                step = 1.0
                h_pred = t.predict(X)
                if self.step_size == "adaptive":
                    step = loss.line_search(y,y_pred,h_pred)

                self.weight[i,k] *= step
                Y_pred[:,k] += self.weight[i,k]*h_pred

        
    def predict(self,X):
        Y_pred = np.zeros((X.shape[0],self.out_dims))
        for i in range(self.n_iter):
            for k in range(self.out_dims):
                Y_pred[:,k] += self.weights[i,k] * self.learners[i,k].predict(X)
                
        if self.classifier:
            Y_pred = Y_pred.argmax(axis = 1)
            
        return Y_pred

## 4. XBGBOOST的算法实现

主要参考资料为印象笔记中陈天奇的[XGBoost 与 Boosted Tree](https://app.yinxiang.com/shard/s43/nl/8933006/3684c27a-1e8d-4c6a-b7f0-8c56a7ca057b/)的文章，这篇文章主要是来源于其在华盛顿大学助教的讲义[陈天奇对于xgboost的解释](https://homes.cs.washington.edu/~tqchen/pdf/BoostedTree.pdf)
同时[Introduction to Boosted Trees](https://xgboost.readthedocs.io/en/latest/tutorials/model.html)xgboost的说明文档也做了就是说明

XGBOOST的理论基础是上面GBDT理论的延续，我们对GBDT的目标函数定义如下

$
Obj^t 
=\sum_{i=1}^{n}l(y_i,\hat{y}_i^{t})
\\=\sum_{i=1}^{n}l(y_i,\hat{y}_i^{t-1}+f_t(X_i))
\\\approx\sum_{i=1}^{n}[l(y_i,\hat{y}_i^{t-1})+\partial_{\hat{y}^{t-1}_{i}} l(y_i,\hat{y}_i^{t-1})f_t(X_i)]
$

xgboost在此的基础上增加了二阶的泰勒展开和正则项的引入

$
Obj^t 
=\sum_{i=1}^{n}l(y_i,\hat{y}_i^{t}) + \sum_{i=1}^{t}\Omega(f_{i})
\\=\sum_{i=1}^{n}l(y_i,\hat{y}_i^{t-1}+f_t(X_i)) + \Omega(f_{t}) + constant
\\\approx\sum_{i=1}^{n}[l(y_i,\hat{y}_i^{t-1})+\partial_{\hat{y}^{t-1}_{i}} l(y_i,\hat{y}_i^{t-1})f_t(X_i)+\frac{1}{2}\partial_{\hat{y}^{t-1}_{i}}^2l(y_i,\hat{y}^{t-1})f_t^2(x_i)] +\Omega(f_{t}) + constant
$

等我们消除常数之后便会得到


$
\sum_{i=1}^{n}[g_if_t(X_i)+\frac{1}{2}h_if_t^2(x_i)] +\Omega(f_{t})
$

$
\\g_i = \partial_{\hat{y}^{t-1}_{i}}l(y_i,\hat{y}_i^{t-1}),
\\h_i = \partial_{\hat{y}^{t-1}_{i}}^2l(y_i,\hat{y}^{t-1})
$

**这就是目前我们经过简化之后的目标函数，不过还没有对复杂度进行定义**

按照陈天奇的定义
> 这个复杂度包含了一棵树里面节点的个数，以及每个树叶子节点上面输出分数的$L2$模平方。

其中
$
f_t(x) = w_q(x), w\in R^T,q:R^d \to\{1,2,3,4,...,T\}
$
直白点的话就是，q(x)指的是输入映射到叶子的索引号，w_i指的是对应的索引号的分数。T是节点的个数
$f_t(x)$ 是叶子节点的分数向量

![定义树](xgboost.png)

$
\Omega(f_{t})=\gamma T + \frac{1}{2}\lambda\sum_{j=1}^{T}w_j^2
$

另外引入每个子叶子上样本的集合
$
I_j = \{i|q(x_i)=j\}
$

所以目标函数可以改写为，陈认为这部**最为关键**

$
Obj^{(t)} \approx \sum_{i=1}^{n}[g_if_t(X_i)+\frac{1}{2}h_if_t^2(x_i)] +\Omega(f_{t})
\\=\sum_{i=1}^{n}[g_iw_{q(x_i)}+\frac{1}{2}h_iw_{q(x_i)}^2] +\gamma T + \frac{1}{2}\lambda\sum_{j=1}^{T}w_j^2
\\=\sum_{j=1}^{T}[(\sum_{i\in I_j}g_i)w_j+\frac{1}{2}(\sum_{i\in I_j}h_i+\lambda)w_j^2] + \gamma T
$

从上面的公式便可以看出，这里做了个转换也就是将每个要计算的数据直接放在叶节点进行计算，从数据输入的角度到最后叶节点的角度

按照陈的说法就是这个目标函数包含了T个相互独立的变量，定义

$G_j = \sum_{i\in I_j}g_i$

$H_j = \sum_{i\in I_j}h_i$

那目标函数便可改写为
$
Obj^{(t)} \approx\sum_{j=1}^{T}[(\sum_{i\in I_j}g_i)w_j+\frac{1}{2}(\sum_{i\in I_j}h_i+\lambda)w_j^2] + \gamma T
\\=\sum_{j=1}^T[G_jW_j+\frac{1}{2}(H_j+\lambda)w_j^2)]+\gamma T
$

也就说我们经过一系列转换之后便得到了简化后的目标函数为了求这个函数的最小值，首先我们要知道的是我们求的是Wj的最有，所以我们直接对目标函数wj的一阶导数等于0，便可得出

$
\sum_{j=1}^T[G_j+(H_j+\lambda)w_j)]=0
$

所以我们最后就得出了最优解和对应的目标函数

$
W_j^* = -\frac{G_j}{(H_j+\lambda)}
\\Obj = -\frac{1}{2}\sum_{j=1}^{T}\frac{G_j^2}{H_j + \lambda} + \gamma T
$

对应的目标函数就像我们的打分函数一样，类似熵和gini系数一样对我们新产生的数进行评估，下图是简单的例子
![The Structure Score Calculation](score.png)

具体到实际的操作环节，主要是用贪心算法对已有的叶子加入一个分割，因为我们用的是CART树，所以我们是按照二叉树那样分割的，计算分割前后的增益

$
Gain = -\frac{1}{2}\frac{G_L+^2+G_R^2}{H_L+H_R+\lambda}+\gamma-(-\frac{1}{2}[\frac{G_L^2}{H_L+\lambda}+\frac{G_R^2}{H_R+\lambda}] + 2\gamma) 
\\=\frac{1}{2}[\frac{G_L^2}{H_L+\lambda}+\frac{G_R^2}{H_R+\lambda}-\frac{G_L+^2+G_R^2}{H_L+H_R+\lambda}] -\gamma
$

这也是我们讲的xgboost与gbdt的主要区别是引入了损失函数的二阶展开和正则项

总结到这里我忽然有疑问既然已经有了熵，gini，mse的评价手段，那为什么还需要梯度来进行函数的优化呢？

这里gini是针对离散变量，cart树输出的是分数所以我们用的是基于连续值的输入进行判定，所以会用mse来衡量，mse只是我们函数的一种特殊形式，我们这个时候定义的目标函数更具有一般性，就像陈所说的
> 这样一个形式包含可所有可以求导的目标函数。也就是说有了这个形式，我们写出来的代码可以用来求解包括回归，分类和排序的各种问题，正式的推导可以使得机器学习的工具更加一般

当然他指的是下面的推导结果

$
\sum_{i=1}^{n}[g_if_t(X_i)+\frac{1}{2}h_if_t^2(x_i)] +\Omega(f_{t})
$

$
\\g_i = \partial_{\hat{y}^{t-1}_{i}}l(y_i,\hat{y}_i^{t-1}),
\\h_i = \partial_{\hat{y}^{t-1}_{i}}^2l(y_i,\hat{y}^{t-1})
$


最后一步的讲解[知乎这篇](https://zhuanlan.zhihu.com/p/29765582)讲的比较清楚

接下来主要就讲算法，由于xgboost源库中的代码超过我自身的能力的范围，而且本文档的主要目的就是对算法的理论和基本实现进行介绍所以没有必要进行深入的介绍

代码实现主要参考知乎的这篇文章[XGBoost的python源码实现](https://zhuanlan.zhihu.com/p/32181687) 当然这个代码的有很多问题但是就基本的核心功能算是实现了，不过具体的评判过程之类的没有那么详细

In [ ]:
from __future__ import division,print_function
import numpy as np
import progressbar 

from decision_tree.decision_tree_model import DecisionTree
from utils.misc import bar_widgets
#这里是对问题的简化，直接定义了均方误差为损失的类，计算其一阶导数二阶导数
#不过感觉这个一阶求导是有问题的，应该是有负号存在的，是对predicted求导
class LeastSquaresLoss():
    def gradient(self,actual,predicted):
        return actual - predicted
    def hess(self,actual,predicted):
        return np.ones_like(actual)
    
class XGboostRegressionTree(DecisionTree):
    def _split(self,y):
        col = int(np.shape(y)[1]/2)
        y,y_pred = y[:,:col],y[:,col:]
        return y,y_pred
    
    def _gain(self,y,y_pred):
        nominator = np.power((self.loss.gradient(y,y_pred)).sum(),2)
        denominator = self.loss.hess(y,y_pred).sum()
        return 0.5*(nominator/denominator)
#这里的增益算法没有1/2的系数和gamma的正则项    
    def _gian_by_taylor(self,y,y1,y2)
        y,y_pred = self._split(y)
        y1,y1_pred = self._split(y1)
        y2,y2_pred = self._split(y2)
        
        true_gain = self._gain(y1,y1_pred)
        false_gain = self._gain(y2,y2_pred)
        gain = self._gain(y,y_pred)
        return true_gain + false_gain - gain
#这里又省略了wi公式中的负号和分母中的lambda
    def _approximate_update(self,y):
        y,y_pred = self._split(y)
        gradient = np.sum(self.loss.gradient(y,y_pred),axis=0)
        hessian = np.sum(self.loss.hess(y,y_pred),axis = 0)
        update_approximation = gradient / hessian
        return update_approximation
        
    def fit(self,X,y):
        self._impurity_calculation = self._gain_by_tayor
        self._leaf_value_calculation = self._approximate_update
        super(XGBoostRegressionTree,self).fit(X,y)
        
class XGBoost(object):
    def __init__(self,n_estimators=200,learning_rate=0.01,min_samples_split=2,min_impurity=1e-7,max_depth=2):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.min_samples_split = min_samples_split
        self.min_impurity = min_impurity
        self.max_depth = max_depth
        
        self.bar = progressbar.ProgressBar(widgets=bar_widgets)
        
        self.loss = LeastSquaresLoss()
        
        self.tresss = []
        
        for _ in range(n_estimators):
            tree = XGBoostRegressionTree(
                    min_samples_split = self.min_samples_split,
                    min_impurity = min_impurity,
                    max_depth = self.max_depth,
                    loss = self.loss)
            self.trees.append(tree)
            
            
    def fix(self,X,y):
        m = X.shape[0]
        y = np.reshape(y,(m,-1))
        y_pred = np.zeros(np.shape(y))
        for i in self.bar(range(self.n_estimators)):
            tree = self.trees[i]
            y_and_pred = np.concatenate((y,y_pred),axis=1)
            tree.fit(X,y_and_pred)
            update_pred = tree.predict(X)
            update_pred = np.reshape(update_pred,(m,-1))
            y_pred += update_pred
            
    def predict(self,X):
        y_pred = None
        m = X.shape[0]
        for tree in self.trees:
            update_pred = tree.predict(X)
            update_pred = np.reshape(update_pred,(m,-1))
            if y_pred is None:
                y_pred = np.zeros_like(update_pred)
            y_pred += update_pred
            
        return y_pred
        

至此便完成了决策树的理论和算法实现，对于底层数学原理的理解有助于对现有的算法进行更加深入的理解，例如那些lightgbm之类在这里都没有讲，个人是觉得适可而止，前面讲的算法也基本覆盖真个决策树的主线。这也是之前我没有做好的事情。后面个人觉得可以按照这种主题式的方式进行学习